# Qiskit Runtime REST API query & job submission

* https://us-east.quantum-computing.cloud.ibm.com/openapi/ 
* General documentation at https://cloud.ibm.com/apidocs/quantum-computing 
* Users needs to [create an account through IBM Cloud Qiskit Runtime Service and access the API key and Cloud Resource Name (CRN)](https://cloud.ibm.com/apidocs/quantum-computing#authentication). 

## Authenticate via IBM Cloud API key and Cloud Resource Name

In [15]:
with open('cloud_token') as file:
    token=file.read()
with open('crn_service') as file:
    crn_service = file.read()

In [23]:
url = 'https://iam.cloud.ibm.com/identity/token'
headers = {'Content-Type': 'application/x-www-form-urlencoded'}
data='grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey='+token
auth_response = requests.post(url, headers= headers, data=data)
auth_id=auth_response.json()['access_token']
auth_type = auth_response.json()['token_type']

## GET available backends

In [17]:
url_backends = 'https://us-east.quantum-computing.cloud.ibm.com/backends'
headers = { 'Content-Type': 'application/json',
            'Service-CRN': crn_service,
            'Authorization':auth_type + ' ' + auth_id}

backends_response = requests.get(url_backends, headers=headers)
backends_response.json()

{'devices': ['ibm_algiers',
  'ibm_brisbane',
  'ibm_kyoto',
  'ibm_osaka',
  'ibmq_qasm_simulator',
  'simulator_stabilizer',
  'simulator_mps',
  'simulator_extended_stabilizer',
  'simulator_statevector']}

In [18]:
backend = backends_response.json()['devices'][0]
backend

'ibm_algiers'

## Run a job

### set up qasm circuit

In [19]:
qasm_string='''
OPENQASM 3;
include "stdgates.inc";
qreg q[1];
creg c[1];
x q[0];
c[0] = measure q[0]; 
'''   

### run circuit via API

In [20]:
import requests

url = 'https://us-east.quantum-computing.cloud.ibm.com/jobs'

headers = { 'Content-Type': 'application/json',
            'Service-CRN': crn_service,
            'Authorization':auth_type + ' ' + auth_id,
            'x-qx-client-application': 'qiskit-version-2/0.39.2/'+'your_application' #specifying the application you might be running from. For an actual Integration project, this option it is invaluable to know where jobs are coming from. At this time the "qiskit-version-2/0.39.2/" string is a necessary prefix.
            }

job_input = {
    'program_id': 'sampler',
    "backend": backend, #Sessions work best with real backends (QCs)
    "start_session": True, #set to False if you just need to run a single job. When addressing simulators, comment out this line. 
    "params": {
        "circuits": [qasm_string]#*5
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code, response.text}")

Job created: {"id":"cmfsca0mogjisssgndv0","backend":"ibm_algiers","session_id":"cmfsca0mogjisssgndv0"}


### Run follow-up jobs in the same Session (optional)

In [21]:
job_input = {
    'program_id': 'sampler',
    "backend": response.json()['backend'],
    "params": {
        "circuits": [qasm_string]*5
}}

if 'session_id' in response.json().keys():
  session_id=response.json()['session_id']
  job_input['session_id'] = session_id
  
response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")

Job created: {"id":"cmfscc0mogjisssgne10","backend":"ibm_algiers","session_id":"cmfsca0mogjisssgndv0"}


### Query Websocket for live job update and close Session

It is very good practice to close a Session when all jobs are done. This will reduce wait time for subsequent Users. So we will listen for a status change of the last job via websocket and close the Session when done.

In [ ]:
import asyncio
import websockets

async def receive_status_updates(api_key):
    uri = "wss://us-east.quantum-computing.cloud.ibm.com/stream/jobs/"+job_id 
    extra_headers = {'Authorization': f'Bearer {api_key}',
                     'Service-CRN': crn_service}

    async with websockets.connect(uri, extra_headers=extra_headers) as websocket:
        while True:
            try:
                message = await websocket.recv()
                print(f"Received message: {message}")  
            except websockets.exceptions.ConnectionClosed:
                print("WebSocket connection closed.")
                break

await receive_status_updates(auth_id)

#subsequently closing Session
status_result = requests.get(url+'/'+job_id,headers=headers)
sessionsurl = 'https://runtime-us-east.quantum-computing.ibm.com/sessions/'+session_id+'/close'

if status_result.json()['state']['status']=='Completed':
   response = requests.delete(sessionsurl,headers=headers)
else:
   print('waiting for job to complete before closing Session '+session_id)

if response.status_code==204:
    print('Session '+session_id+' was closed')

### Get results

In [28]:
response_result= requests.get(url+'/'+job_id+'/results', headers=headers)

response_result.json()

{'quasi_dists': [{'0': 0.002374284670644156, '1': 0.997625715329356},
  {'0': -0.002191647388286806, '1': 1.0021916473882868},
  {'0': -0.002445310280449637, '1': 1.0024453102804496},
  {'0': 0.003896262023621144, '1': 0.9961037379763789},
  {'0': -0.0009233329274726501, '1': 1.0009233329274727}],
 'metadata': [{'shots': 4000,
   'circuit_metadata': {},
   'readout_mitigation_overhead': 1.0333924314440097,
   'readout_mitigation_time': 0.038907336071133614},
  {'shots': 4000,
   'circuit_metadata': {},
   'readout_mitigation_overhead': 1.0333924314440097,
   'readout_mitigation_time': 0.0004875538870692253},
  {'shots': 4000,
   'circuit_metadata': {},
   'readout_mitigation_overhead': 1.0333924314440097,
   'readout_mitigation_time': 0.0003698021173477173},
  {'shots': 4000,
   'circuit_metadata': {},
   'readout_mitigation_overhead': 1.0333924314440097,
   'readout_mitigation_time': 0.001236913725733757},
  {'shots': 4000,
   'circuit_metadata': {},
   'readout_mitigation_overhead': 